In [1]:
import whisper
import torch
from pydub import AudioSegment
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "|1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import base64
from io import *
import struct
import numpy as np
import tempfile

Temporary directory: /tmp


In [ ]:
def extract_wav_header(binary_data):
    # WAV 파일 헤더 구조에 맞게 필드를 읽음
  
    chunk_id = binary_data[:4]
    chunk_size = struct.unpack('<I', binary_data[4:8])[0]
    format_str = binary_data[8:12]
    subchunk1_id = binary_data[12:16]
    subchunk1_size = struct.unpack('<I', binary_data[16:20])[0]
    audio_format = struct.unpack('<H', binary_data[20:22])[0]
    num_channels = struct.unpack('<H', binary_data[22:24])[0]
    sample_rate = struct.unpack('<I', binary_data[24:28])[0]
    byte_rate = struct.unpack('<I', binary_data[28:32])[0]
    block_align = struct.unpack('<H', binary_data[32:34])[0]
    bits_per_sample = struct.unpack('<H', binary_data[34:36])[0]
    subchunk2_id = binary_data[36:40]
    subchunk2_size = struct.unpack('<I', binary_data[40:44])[0]
    

    header = {"Chunk_ID": chunk_id,
      "Chunk_Size" : chunk_size,
      "Format" : format_str,
      "Subchunk1_ID" : subchunk1_id,
      "Subchunk1_Size" : subchunk1_size,
      "Audio_Format" : audio_format,
      "Num_Channels" : num_channels,
      "Sample_Rate" : sample_rate,
      "Byte_Rate" : byte_rate,
      "Block_Align" : block_align,
      "Bits_Per_Sample" : bits_per_sample,
      "Subchunk2_ID" : subchunk2_id,
      "Subchunk2_Size" : subchunk2_size}
    return header 

def make_header(wav_header, input_data_size) :
    chunk_id = wav_header['Chunk_ID']
    chunk_size = input_data_size + 44  # 데이터 크기에 추가 정보 크기(46바이트)를 더함
    format_str = wav_header['Format']
    subchunk1_id = wav_header['Subchunk1_ID']
    subchunk1_size = wav_header['Subchunk1_Size']  # 일반적인 값을 사용함
    audio_format = wav_header['Audio_Format'] # PCM 포맷
    num_channels = wav_header['Num_Channels']  # 스테레오
    sample_rate = wav_header['Sample_Rate']  # 예시로 44100Hz
    byte_rate = wav_header['Byte_Rate']
    block_align = wav_header['Block_Align']    # 채널 수 * 샘플 크기(2바이트)
    bits_per_sample = wav_header['Bits_Per_Sample']   # 16비트 PCM 사용
    subchunk2_id =  b'data'
    subchunk2_size = input_data_size + 44  # 데이터 크기

    wav_header = struct.pack('<4sI4s4sIHHIIHH4sI', chunk_id, chunk_size, format_str, subchunk1_id,
                            subchunk1_size, audio_format, num_channels, sample_rate, byte_rate,
                            block_align, bits_per_sample, subchunk2_id, subchunk2_size)
    
    return wav_header

def STT_module(audio_with_header,model) :
    # 임시 파일 생성
    temp_file = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
    temp_file.write(audio_with_header)
    temp_path = temp_file.name

    # STT 실행
    result = model.transcribe(temp_path, fp16=False)
    print(result['text'])

    # 임시 파일 삭제
    os.remove(temp_path)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 메인 모델
model = whisper.load_model("medium")
model = model.to(device)

In [3]:
file_path = "/home/jongho/바탕화면/테스트 음성파일/"
file_name = "연제리2.wav"

encode_audio = base64.b64encode(open(f"{file_path}{file_name}", "rb").read()).decode('utf-8') 

In [5]:
# 웹 소켓을 통해서 base64 데이터를 수신 받았을때
# 바이너리 데이터로 변환
decoded_audio = base64.b64decode(encode_audio)

# 오디오 데이터에서 헤더정보 추출
BINARY_DATA = decoded_audio[:44]
audio_signal_data = decoded_audio[44:]

wav_header = extract_wav_header(BINARY_DATA)

print("음성파일 헤더 정보 :",wav_header)

음성파일 헤더 정보 : {'Chunk_ID': b'RIFF', 'Chunk_Size': 543172920, 'Format': b'WAVE', 'Subchunk1_ID': b'fmt ', 'Subchunk1_Size': 16, 'Audio_Format': 1, 'Num_Channels': 2, 'Sample_Rate': 44100, 'Byte_Rate': 176400, 'Block_Align': 4, 'Bits_Per_Sample': 16, 'Subchunk2_ID': b'LIST', 'Subchunk2_Size': 80}


In [6]:
# 분할 사이즈
SPLIT_SIZE = 10000000 # 10MB
# 기본 경로
BASE_PATH = "/home/jongho/Spring_project/CJU_STT/CJU_Web_Application_Services/src/main/webapp/resources/python/result"
# 분할 바이너리 데이터 리스트
split_data = []

for i in range(0, len(audio_signal_data), SPLIT_SIZE):
    split_data.append(audio_signal_data[i:i+SPLIT_SIZE])

print(f"바이너리 데이터 분할 완료 : {len(split_data)}개")

바이너리 데이터 분할 완료 : 55개


In [7]:
for i in range(len(split_data)) :
    # WAV 파일 헤더 정보 생성
    audio_with_header = make_header(wav_header, len(split_data[i])) + split_data[i]
   
    # API 처리 구간
    STT_module(audio_with_header, model)

/home/jongho/.local/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/home/jongho/.local/lib/python3.8/site-packages/whisper/decoding.py:750: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  [t[self.sample_begin : (t == tokenizer.eot).nonzero()[0, 0]] for t in s]


 해수 받으면 초기화하고 2개 대사 검거를 한 다음에 초기화하고 다음 조사를 위해서 세팅하는 거 네 개거든요 8개씩 해서 32개 하시면 되고 충전하고 배포하는 거 해주시면 되고요 조사 물품도 곤약에 물품 수정에 맞게 배포해주시면 되세요 그래서 지금 아까 조사 일정이 따라서 저희가 배포를 하고 다음 날부터 착용을 하게 하잖아요 착용하는 날에 문자 알림을 보내실 거죠 그리고 중간연검 3일째 그리고 마지막 회수하는 시절에 한 3번 정도 알림을 보내시는 걸로 알고 있고 그리고 저희가 조사원하고 업무를 어떻게 해야 될지를 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아
 프로토컬을 이렇게 만드셔가지고 그걸 하나 딱 표준화해서 만들어놓으면 좋을 것 같거든요 그래서 저희가 오늘 말씀드린 내용으로 뭔가 만들어서 보내주시면 네, 확정해서 진행하면 될 것 같아요 교육이나 뭐 이런 거 할 때도 딱 그렇게 진행하면 될 것 같아서 네, 그리고 3프레이지에서는 이제 교상자 확인하고 관리 분단자를 지정해야지 되는 거 그... 지금... 저희가 침이 내 팀이라고 했잖아요 그러면은 담당하시는 분은 팀별로 배공을 하실 건가요? 아니면 한 분이 내 팀을 다 